In [ ]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random


In [ ]:
from collections import deque

def BFS(board, start_row, start_col):
  dr = [0,-1,0,1,0]
  dc = [0,0,1,0,-1]

  result = 0
  if start_row == 1 and start_col == 1:
    result = -1
  queue = deque([])
  visited = [[False]*5 for i in range(5)]
  
  visited[start_row][start_col] = True
  queue.append((start_row,start_col))

  while(len(queue) != 0):
    (row,col) = queue.popleft()
    result = result + 1

    for i in range(5):
      nr = row + dr[i]
      nc = col + dc[i]
      if nr < 0 or nr >= 5 or nc < 0 or nc >= 5:
        continue
      if board[nr][nc] == 1:
        continue
      if visited[nr][nc] == True:
        continue

      visited[nr][nc] = True
      board[nr][nc] = 1
      queue.append((nr,nc))
  
  return result


temp = [[0]*5 for i in range(5)]

temp[1][1] = 1
BFS(temp,1,1)


24

In [ ]:
class BingoEnv(Env):
  def __init__(self):
    self.round = 0
    self.action_space = Discrete(25)
    self.observation_space = Box(low=0, high=2, shape=(15,5,5), dtype=np.int32)
    self.state = np.zeros((15,5,5))
    self.first_rand_pos = random.choice([i for i in range(0,25) if i not in [6]])
    self.second_rand_pos = random.choice([i for i in range(0,25) if i not in [6,self.first_rand_pos]])
    self.first_rand_pos_row = self.first_rand_pos // 5
    self.first_rand_pos_col = self.first_rand_pos % 5
    self.second_rand_pos_row = self.second_rand_pos // 5
    self.second_rand_pos_col = self.second_rand_pos % 5


  def check_bingo(board, row,col):
    result_list = []
    if board[row][col] == 2:
      return []

    result = 1
    for i in range(5):
      result = result * board[row][i]
      if result == 0:
        break
    if result > 0:
      result_list.append("ROW")
    
    result = 1
    for i in range(5):
      result = result * board[i][col]
      if result == 0:
        break
    if result > 0:
      result_list.append("COL")

    if row == col:
      result = 1
      result = result * board[0][0] * board[1][1] * board[2][2] * board[3][3] * board[4][4]
      if result > 0:
        result_list.append("LC")

    if row+col == 4:
      result = 1
      result = result * board[0][4] * board[1][3] * board[2][2] * board[3][1] * board[4][0]
      if result > 0 :
        result_list.append("RC")

    return result_list

  def step(self, action):
    self.round = self.round+1
    #cur_board = self.state[self.round]
    row = action // 5
    col = action % 5
    bingo_count = 0
    canClick = False

#None Up Right Down Left
    dr = [0,-1,0,1,0]
    dc = [0,0,1,0,-1]

    for i in range(5):
      nr = row + dr[i]
      nc = col + dc[i]
      if nr < 0 or nr >= 5 or nc < 0 or nc >= 5:
        continue

      if self.state[self.round][nr][nc] == 2:
        continue

      elif self.state[self.round][nr][nc] == 1:
        self.state[self.round][nr][nc] = 0
        

      elif self.state[self.round][nr][nc] == 0:
        self.state[self.round][nr][nc] = 1
        isBingo = self.check_bingo(self.state[self.round],nr,nc)
        bingo_count = bingo_count + len(isBingo)

        for dir in isBingo:
          if dir == "ROW":
            for i in range(5):
              self.state[self.round][nr][i] = 2
          elif dir == "COL":
            for i in range(5):
              self.state[self.round][i][nc] = 2
          elif dir == "LC":
            self.state[self.round][0][0] = 2
            self.state[self.round][1][1] = 2
            self.state[self.round][2][2] = 2
            self.state[self.round][3][3] = 2
            self.state[self.round][4][4] = 2
          elif dir == "RC":
            self.state[self.round][0][4] = 2
            self.state[self.round][1][3] = 2
            self.state[self.round][2][2] = 2
            self.state[self.round][3][1] = 2
            self.state[self.round][4][0] = 2
    
    
    around_block_num = BFS(self.state[self.round],1,1)
    if self.round % 3 == 0 and bingo_count == 0:
      reward = -1
    elif self.round % 3 == 0 and bingo_count > 0:
      reward = 1 + around_block_num / 50

    board_sum = 0
    for i in range(5):
      for j in range(5):
        board_sum = board_sum + self.state[self.round][i][j]

    if (board_sum == 50) or self.round == 14:
      done = True
    else:
      done = False
    
    info ={}
    return self.state, reward, done, info

  def render(self):
    print(self.state[self.round])


    

t = BingoEnv()



In [ ]:
True and True

True